In [ ]:
%pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
%pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes
%pip install xformers[cuda] -U
%pip install wandb

#Weights and Bias Login


In [ ]:
import json
import wandb
with open('wandb_config.json') as f:
    config = json.load(f)
wandb.login(key=config['api_key'])

FileNotFoundError: [Errno 2] No such file or directory: 'wandb_config.json'

In [ ]:
import wandb
wandb.login(key="33b1bc53e217a2129ce5cb5d04e10dcd5f743f5c")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

#Below Models are already quantized by unsloth.

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit",
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit"
]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-bnb-4bit",
    #model_name="unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

#'r' refers to dimensionality reduction wherein we use mutliples of 8.
#Target Modules are the factors being reduced.

In [ ]:
  model = FastLanguageModel.get_peft_model(
    model,
    r=8,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

# Hugging Face's Alpaca Dataset
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN # EOS Token is needed else there is no limit.
        texts.append(text)
    return {"text": texts}

from datasets import load_dataset

dataset = load_dataset("yahma/alpaca-cleaned", split="train")
dataset = dataset.map(formatting_prompts_func, batched=True)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Define trainer arguments
trainer_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=60, # If Max Steps is given, Epochs are overridden
    #num_train_epochs=5, # Currently Overridding Epochs because of large execution time
    learning_rate=2e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs",
    report_to="wandb"
)
# Initialize SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=trainer_args
)

Map (num_proc=2):   0%|          | 0/51760 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.535 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 51,760 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 20,971,520
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: oxalis (oxalis_1). Use `wandb login --relogin` to force relogin


Step,Training Loss
1,1.814500
2,2.285700
3,1.686600
4,1.940000
5,1.636200
6,1.589600
7,1.181400
8,1.255500
9,1.108000
10,1.161700


In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [ ]:
def generate_language_model_output(instruction, model, tokenizer, alpaca_prompt, input=None, max_tokens=128):
    FastLanguageModel.for_inference(model)
    prompt = alpaca_prompt.format(instruction, input, "")
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=max_tokens, use_cache=True)
    decoded_outputs = tokenizer.batch_decode(outputs)
    return decoded_outputs

while True:
    instruction = input("Enter your question (or 'quit' to exit): ")
    if instruction.lower() == 'quit':
        break
    generated_output = generate_language_model_output(instruction, model, tokenizer, alpaca_prompt)
    response = generated_output[-1].split("### Response:\n")[-1].strip()
    print(f"Question: {instruction}")
    print(f"Answer: {response}")

Enter your question (or 'quit' to exit): what is alcohol percentage in gin?
Question: what is alcohol percentage in gin?
Answer: The alcohol percentage in gin typically ranges from 36% to 50%, with most gins falling somewhere in between. Gin is a spirit distilled from a base of grain or other botanicals, which gives it its characteristic flavor and high alcohol content.<|end_of_text|>
Enter your question (or 'quit' to exit): How to make paneer butter masala?
Question: How to make paneer butter masala?
Answer: Paneer butter masala is a classic Indian dish made with cubes of paneer (Indian cottage cheese) in a rich, creamy tomato-based sauce. Here is a step-by-step guide on how to make it:

1. Cut the paneer into 1-inch cubes and keep aside.
2. Heat oil in a pan and add chopped onions, garlic, and ginger. Saute until the onions are translucent.
3. Add chopped tomatoes and cook until they become mushy.
4. Add red chili powder, coriander powder, cumin powder, turmeric powder, and garam mas

# SPEECH AS OUTPUT

# Google TTS

In [ ]:
!pip install gtts
!pip install SpeechRecognition

In [ ]:
from gtts import gTTS
from IPython.display import display, Audio, clear_output
import time

def generate_language_model_output(instruction, model, tokenizer, alpaca_prompt, input=None, max_tokens=128):
    FastLanguageModel.for_inference(model)
    prompt = alpaca_prompt.format(instruction, input, "")
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=max_tokens, use_cache=True)
    decoded_outputs = tokenizer.batch_decode(outputs)
    return decoded_outputs

def text_to_speech(text, filename="response.mp3"):
    tts = gTTS(text)
    tts.save(filename)
    return Audio(filename, autoplay=True)

while True:
    instruction = input("Enter your question (or 'quit' to exit): ")
    if instruction.lower() == 'quit':
        break  # Exit the loop if 'quit' is entered

    generated_output = generate_language_model_output(instruction, model, tokenizer, alpaca_prompt)
    response = generated_output[-1].split("### Response:\n")[-1].strip()

    # Remove the last part of the response
    response = response.split(",")[0]  # Adjust as needed based on response structure

    print(f"Question: {instruction}")
    print(f"Answer: {response}")

    audio_display = text_to_speech(response)
    display(audio_display)

    # Wait for audio playback (a rough estimate, adjust as needed)
    time.sleep(len(response) / 15)  # Adjust the divisor based on average reading speed

Enter your question (or 'quit' to exit): Who is Charlie Puth?


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: Who is Charlie Puth?
Answer: Charlie Puth is an American singer


Enter your question (or 'quit' to exit): Python Code to Print a rectangle of stars


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: Python Code to Print a rectangle of stars
Answer: ```python
print("+" + "---" * 5 + "+")
print("|" + " " * 5 + " " * 5 + "|")
print("|" + " " * 5 + " " * 5 + "|")
print("|" + " " * 5 + " " * 5 + "|")
print("+" + "---" * 5 + "+")
```<|end_of_text|>


Enter your question (or 'quit' to exit): quit


In [ ]:
import speech_recognition as sr
from google.colab import files
from gtts import gTTS
from IPython.display import display, Audio, clear_output
import time
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Function to upload audio file
def upload_file():
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]
    return filename

# Function to convert speech to text
def speech_to_text(filename):
    recognizer = sr.Recognizer()
    with sr.AudioFile(filename) as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data)
            print(f"Recognized text: {text}")
            return text
        except sr.UnknownValueError:
            print("Google Speech Recognition could not understand the audio")
            return ""
        except sr.RequestError as e:
            print(f"Could not request results from Google Speech Recognition service; {e}")
            return ""

# Function to generate language model output
def generate_language_model_output(instruction, model, tokenizer, max_tokens=128):
    inputs = tokenizer(instruction, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=max_tokens, use_cache=True)
    decoded_outputs = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded_outputs

# Function to convert text to speech
def text_to_speech(text, filename="response.mp3"):
    tts = gTTS(text)
    tts.save(filename)
    return Audio(filename, autoplay=True)

# Upload and process the audio file
filename = upload_file()
question_text = speech_to_text(filename)

# Ensure valid text is returned from speech recognition
if question_text:
    # Load language model and tokenizer (adjust the model name as needed)
    model_name = "distilgpt2"  # Example model, adjust as needed
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")

    # Generate response
    response_text = generate_language_model_output(question_text, model, tokenizer)
    response = response_text.split("### Response:\n")[-1].strip()

    print(f"Question: {question_text}")
    print(f"Answer: {response}")

    # Convert the response to speech and play it
    audio_display = text_to_speech(response)
    display(audio_display)

    # Wait for audio playback (a rough estimate, adjust as needed)
    time.sleep(len(response) / 15)  # Adjust the divisor based on average reading speed
else:
    print("No valid question text recognized from the audio.")

Saving A_3.wav to A_3 (1).wav
Recognized text: it is logical that the United States should do whatever it is able to do to assist in the return of normal economic health in the world without which there can be no political stability and no assured peace our policy is directed not against any country or doctrine but again desperation


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question: it is logical that the United States should do whatever it is able to do to assist in the return of normal economic health in the world without which there can be no political stability and no assured peace our policy is directed not against any country or doctrine but again desperation
Answer: it is logical that the United States should do whatever it is able to do to assist in the return of normal economic health in the world without which there can be no political stability and no assured peace our policy is directed not against any country or doctrine but again desperation.



The United States should not be a party to the international order. It is a party to the international order. It is a party to the international order. It is a party to the international order. It is a party to the international order. It is a party to the international order. It is a party to the international order. It is a party to the international order. It is a party to the international order